In [ ]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import RegularPolygon

# Update the animation function to plot each station as a triangle
def update_plot(frame_number, station_coords, ax, time_text, norm):
    ax.clear()
    ax.set_xlim(station_coords[:, 0].min() - 1000, station_coords[:, 0].max() + 1000)
    ax.set_ylim(station_coords[:, 1].min() - 1000, station_coords[:, 1].max() + 1000)
    ax.set_title('Seismic Event Detections')
    detection_counts = filtered_events_df.iloc[frame_number, 1:].fillna(0).values

    # Use log scale to represent the number of detections; add 1 to avoid log(0)
    colors = np.log(detection_counts + 1)

    # Create triangle patches for each station
    patches = []
    for (x, y), color in zip(station_coords, colors):
        triangle = RegularPolygon((x, y), numVertices=3, radius=200, orientation=np.pi/2, facecolor=plt.cm.inferno(norm(color)))
        patches.append(triangle)
    
    # Add patches to the axes
    p = PatchCollection(patches, match_original=True)
    ax.add_collection(p)
    time_text.set_text(filtered_events_df['hour'][frame_number])

    # Only create colorbar for the first frame
    if frame_number == 0:
        sm = plt.cm.ScalarMappable(cmap='inferno', norm=norm)
        sm.set_array([])  # You have to set a dummy array for the ScalarMappable
        fig.colorbar(sm, ax=ax, orientation='vertical', label='Log(Number of Detections + 1)')

# Define the normalization for color mapping based on all detection counts
all_detection_counts = filtered_events_df.iloc[:, 1:].values.flatten()
norm = plt.Normalize(np.log(all_detection_counts[all_detection_counts > 0].min()), np.log(all_detection_counts.max()))

# Create the plot and the animation with the updated function
fig, ax = plt.subplots()
ax.set_aspect('equal')
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

ani = animation.FuncAnimation(fig, update_plot, frames=n_frames, fargs=(ordered_station_coords, ax, time_text, norm),
                              interval=100, blit=False)

plt.close()  # Prevents duplicate display

# Display the animation
HTML(ani.to_jshtml())
